In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from ipywidgets import interact

# from conc_obj import EEGData
from eegdata_multi import EEGData
from utils.plt import plot_psd, plot_montage
from utils.ica import plot_ica_comp

# MNE imports
import mne
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

import json


***Macros***

>General use macros, importing JSON files to use as the configuration files

In [ ]:
script_path = Path().resolve()
folder = (script_path / "../").resolve()

JSON_MAIN_PATH = script_path / "config/config_main.json"
JSON_CSP_PATH = script_path / "config/config_csp.json"
JSON_GRID_PATH = script_path / "config/config_grid.json"
EVENTS_PATH = script_path / "config/events.json"

with open(JSON_MAIN_PATH, "r") as f:
    config_main = json.load(f)

with open(JSON_CSP_PATH, "r") as f:
    config_csp = json.load(f)
    
with open(JSON_GRID_PATH, "r") as f:
    json_grid = json.load(f)

VERBOSE = config_main['verbose'].lower() == 'true'

L_FREQ = config_main['l_freq']
H_FREQ = config_main['h_freq']

N_SUBJECTS = config_main["n_subjects"]
N_COMPONENTS = config_main["n_components"]

"""
T0 corresponds to rest
T1 corresponds to onset of motion (real or imagined) of
the left fist (in runs 3, 4, 7, 8, 11, and 12)
both fists (in runs 5, 6, 9, 10, 13, and 14)
T2 corresponds to onset of motion (real or imagined) of
the right fist (in runs 3, 4, 7, 8, 11, and 12)
both feet (in runs 5, 6, 9, 10, 13, and 14)
"""


***Initialization of EEG object***

>***(If the files are not locally stored, it will download them to the user system automatically)***

>***Use of functions like .filter_data() also is obligatory if there is no data stored locally***

In [ ]:
eeg_obj = EEGData(config_main, config_csp, folder, verbose=VERBOSE)

#* Filters data and plots PSD to see differences
# eeg_obj.filter_data()
# eeg_obj.plot_psd_ba_filt(verbose=VERBOSE)

# eeg_obj.plot_psd(verbose=VERBOSE)

#* Normalizes data
# eeg_obj.normalize_data()

**Basic information and montage plotting in 2D & 3D**
> ***The channel names can also be printed***

In [ ]:

#* Plots different montages in 2D & 3D
# data = eeg_obj.get_raw_h()
# ch_names = data.info["ch_names"] 
# plot_montage(eeg_obj.montage, ch_names)

**ICA(Independent Component Analysys)**
> ***The number of components that ICA will try to sort out can be changed, it is advised to use values in the range [15-45]***

> ***Ocular artifacts are also removed, since they don't contribute to the muscular movement on this evaluation***

> ***The components can also be plotted and ocular artifacts, EOG, will be clearly visible***

In [ ]:

#* Computes ICA components
# eeg_obj.decomp_ica(n_components=N_COMPONENTS, plt_show=True, verbose=VERBOSE)

#* Plot components of ICA
# plot_ica_comp(folder / config["path_ica_h"])

In [ ]:
data, _ = eeg_obj.get_filt()
events, _ = eeg_obj.get_events()
event_l = config_csp["ev_mlist_eight"]
groupeve_dict = config_csp["event_dict_h"]
event_dict1 = {key: value for key, value in groupeve_dict.items() if value in event_l[0]}
print(event_dict1)

epochs = mne.Epochs(data, events, event_id=event_dict1, tmin=0.3, tmax=3.3, baseline=None, verbose=VERBOSE)
data = epochs.get_data()
# data = data.reshape(data.shape[0], -1)

labels = epochs.events[:, -1]
print(data.shape, labels.shape)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svm_clf = SVC(kernel='rbf', C=100, gamma=2, probability=True)
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42)

ensemble = VotingClassifier(estimators=[('svm', svm_clf), ('rf', rf_clf)], voting='soft')

# Create a new pipeline for LDA
pipeline = Pipeline([
    # ('ica', FastICA(n_components=8)), ('pca')
    ('csp', CSP(n_components=12, log=True, norm_trace=False)),
    ('scaler', StandardScaler()), # StandardScaler works best
	('voting_cs', ensemble)
	# ('lda', LDA())
    # ('svm', SVC(kernel='rbf', C=100, gamma=2))
	# ('rf', RandomForestClassifier())
])

# Fit the 3D pipeline to the transformed data
# pipeline.fit(data, labels)


***GridSearch - Parameter selection***

>Exhaustive search over specified parameter values for an estimator.

In [ ]:
from utils.gridsearch import grid_finder, grid_search

# grid = grid_finder(json_grid, 'svm', 'wide')
# print(grid)
# grid_search(data, labels, pipeline, grid)

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, data, labels, cv=cv)
print("Cross-validation scores:", scores)

**Saving data locally**
> ***Saving and loading data locally saves time and computational power***

> ***Note that it is also needed to change the configuration at the JSON files to import local files***

In [ ]:
#* Saves filtered and concatenated data for faster loading
# eeg_obj.save_type_data(type="raw", folder_path=folder)
# eeg_obj.save_type_data(type="filtered", folder_path=folder)
# eeg_obj.save_type_data(type="norm", folder_path=folder)
# eeg_obj.save_type_data(type="ica", folder_path=folder)